# Call Mallam model

In [1]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Create pipeline
pipe = pipeline("text-generation", model="mesolitica/mallam-1.1B-4096")

# Generate text
output = pipe("Apa khabar?", max_length=100, do_sample=True)
print(output[0]['generated_text'])

# Alternatively, use model and tokenizer directly
tokenizer = AutoTokenizer.from_pretrained("mesolitica/mallam-1.1B-4096")
model = AutoModelForCausalLM.from_pretrained("mesolitica/mallam-1.1B-4096")

inputs = tokenizer("Apa khabar?", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, do_sample=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


2025-04-20 08:13:07.321386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745136787.508826      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745136787.563936      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Apa khabar? Dah lama saya tidak menulis di blog ini, dah lama dah saya tidak ada cerita baharu untuk di kongsikan bersama. Saya bukan apa. Saya dah penat menulis di sini kerana tidak ada kesudahan. Entah mengapa, saya tidak mampu meninggalkannya. Mungkin juga juga, kerana faktor jarak rumah dengan pusat pengajian menyebabkan saya jarang berada di rumah, maka blog juga akan terbiar tanpa di kemaskini. Jika rajin, saya akan update kembali. Kalau malas juga, itu bukan lagi rahsia.\nPost


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Apa khabar? Saya harap anda seronok kerana dapat membaca lebih lanjut mengenai cara-cara membuat dan menggunakan Google Docs sendiri, semoga!\nApakah itu Google Docs? Google Docs adalah aplikasi perkhidmatan yang digunakan untuk membangunkan dokumen di laman web. Ia mudah digunakan dan mempunyai ciri yang menarik untuk meningkatkan kemudahan anda.\nDi Google Docs, anda boleh mengembangkan sebarang jadual di mana anda bekerja. Dalam kes ini, anda hanya perlu menjalankan Google Docs di kawasan pentadbir dan menggunakan fungsi yang sama di


# Evaluation for translation task

# Step 1: Prepare a Translation Dataset

In [3]:
test_data = [
    {"input": "Hello, how are you?", "expected": "Hai, apa khabar?"},
    {"input": "I love to eat nasi lemak.", "expected": "Saya suka makan nasi lemak."},
    {"input": "Where is the nearest hospital?", "expected": "Di manakah hospital yang terdekat?"},
]


# Step 2: Prompt the Model to Translate

In [4]:
from transformers import pipeline

pipe = pipeline("text-generation", model="mesolitica/mallam-1.1B-4096")

results = []
for sample in test_data:
    prompt = f"Translate to Malay: {sample['input']}"
    output = pipe(prompt, max_length=100, do_sample=False)[0]['generated_text']
    results.append({
        "input": sample["input"],
        "expected": sample["expected"],
        "output": output.strip()
    })


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


#  Step 3: Evaluate the Output

In [5]:
!pip install evaluate
import evaluate

bleu = evaluate.load("bleu")

predictions = [r["output"] for r in results]
references = [[r["expected"]] for r in results]  # Note: nested list for BLEU

score = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {score['bleu']:.4f}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

BLEU Score: 0.0000


In [6]:
for r in results:
    print(f"\nInput: {r['input']}")
    print(f"Expected: {r['expected']}")
    print(f"Model Output: {r['output']}")



Input: Hello, how are you?
Expected: Hai, apa khabar?
Model Output: Translate to Malay: Hello, how are you? I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine. I'm fine.

Input: I love to eat nasi lemak.
Expected: Saya suka makan nasi lemak.
Model Output: Translate to Malay: I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat nasi lemak. I love to eat

Input: Where is the nearest hospital?
Expected: Di manakah hospital yang terdekat?
Model Output: Translate to Malay: Where is the nearest hospital?\nPosted by Mohd.Noor Azam bin Atan at 10:00 AM No comments:\nPos

# Step-by-Step: Access Layers, Attention, and Model Internals

# Load the model manually (not with pipeline)

# Inspect Model Architecture Without Weights

In [7]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("mesolitica/mallam-1.1B-4096")

print(config)


MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32000
}

